In [ ]:
# import everything
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras import Sequential, optimizers
from keras.layers import Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard, EarlyStopping
import time


np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
# load the data from numpy objects
X_train = np.load('../my_dataset/X_train.npy')
X_test = np.load('../my_dataset/X_test.npy')
X_validate = np.load('../my_dataset/X_validate.npy')

y_train = np.load('../my_dataset/y_train.npy')
y_test = np.load('../my_dataset/y_test.npy')
y_validate = np.load('../my_dataset/y_validate.npy')

In [ ]:
conv_layers = 3
conv_nodes = 48
epochs = 200
early_stops = 6
repeats = 5

dropout = 0.25
batches = [16, 32, 64, 128, 256]
# equal dropout
for batch in batches:
    for i in range (repeats):
        NAME = "model-{}-batch-{}".format(i, batch)
        print(NAME)

        model = Sequential()

        model.add(Conv2D(conv_nodes, (3, 3), input_shape=X_train.shape[1:]))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout))

        for l in range(conv_layers-1):
            model.add(Conv2D(conv_nodes, (3, 3)))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            model.add(Dropout(dropout))

        model.add(Flatten())

        # output layer (as big as the number of words we teach)
        output_neurons = len(np.unique(y_train))
        model.add(tf.keras.layers.Dense(output_neurons, activation=tf.nn.softmax))

        tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))


        model.compile(loss='sparse_categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'],
                    )

        callbacks = [EarlyStopping(monitor='val_accuracy', patience=early_stops, restore_best_weights=True), tensorboard]

        model.fit(X_train, y_train,
                epochs=epochs,
                validation_data=(X_validate, y_validate),
                callbacks=callbacks,
                batch_size=batch)

            # save the model
        model.save("models/"+NAME)

In [ ]:
accuracies = np.zeros((len(batches), repeats))
average_accuracies = np.zeros(len(batches))

index=0
for batch in batches:
    for i in range (repeats):
        NAME = "model-{}-batch-{}".format(i, batch)
        model = tf.keras.models.load_model("models/"+NAME)
        accuracies[index][i] = round(model.evaluate(X_validate, y_validate)[1] * 100, 2)
        average_accuracies[index] += accuracies[index][i]
    index+=1

average_accuracies = average_accuracies/5

In [ ]:
print("|Batch size| Model 0 | Model 1 | Model 2 | Model 3 | Model 4 | Average |")
print("| -------- | ------- | ------- | ------- | ------- | ------- | ------- |")
index=0
for batch in batches:
    print("| Batch "+str(batches[index])+" | "+str(accuracies[index][0])+" | "+str(accuracies[index][1])+" | "+str(accuracies[index][2])+
    " | "+str(accuracies[index][3])+" | "+str(accuracies[index][4])+" | "+ str(average_accuracies[index])+" | ")
    index+=1

|Batch size| Model 0 | Model 1 | Model 2 | Model 3 | Model 4 | Average |
| -------- | ------- | ------- | ------- | ------- | ------- | ------- |
| Batch 16 | 92.5 | 92.5 | 93.75 | 92.5 | 93.12 | 92.874 | 
| Batch 32 | 90.62 | 94.38 | 91.87 | 93.12 | 93.12 | 92.622 | 
| Batch 64 | 91.25 | 93.12 | 91.87 | 90.62 | 93.75 | 92.122 | 
| Batch 128 | 91.87 | 91.25 | 90.0 | 95.63 | 91.25 | 92.0 | 
| Batch 256 | 86.87 | 91.87 | 91.87 | 91.25 | 89.38 | 90.248 | 